In [ ]:
import os
import sys
import argparse

import toast
from toast.mpi import MPI

import toast.io
import toast.schedule
import toast.ops

In [ ]:
toast.__version__

In [ ]:
env = toast.utils.Environment.get()
log = toast.utils.Logger.get()

In [ ]:
# Get optional MPI parameters
comm, procs, rank = toast.mpi.get_world()

In [ ]:
# Get just our focalplane and schedule from the command line
#! Work only for Python script
parser = argparse.ArgumentParser(description="Simple Ground Simulation Example.")

parser.add_argument(
    "--focalplane", required=True, default=None, help="Input fake focalplane"
)

parser.add_argument(
    "--schedule", required=True, default=None, help="Input observing schedule"
)

parser.add_argument(
    "--thinfp",
    required=False,
    default=1,
    help="Focalplane thinning factor",
    type=int,
)

parser.add_argument(
    "--weather",
    required=False,
    default=None,
    help="Weather information. Infer from observing site if not provided.",
)

In [ ]:
# Build the parser
args = parser.parse_args(['--focalplane', '/home/laclavere/Documents/Thesis/qubic/qubic/scripts/Atmosphere/focalplanes/focalplane_SAT1_SAT_f095.h5',
            '--schedule', '/home/laclavere/Documents/Thesis/qubic/qubic/scripts/Atmosphere/schedules/pole_schedule_sat.1ces.txt',
            '--thinfp', '1'])


In [ ]:
# Create our own directory
out_dir = "toast_sim_ground_simple"
if comm is not None or rank == 0:
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    else:
        print("Output directory {} already exists".format(out_dir))

In [ ]:
 # Load a generic focalplane file.
focalplane = toast.instrument.Focalplane(thinfp=args.thinfp)
with toast.io.H5File(args.focalplane, "r", comm=comm, force_serial=True) as f:
    focalplane.load_hdf5(f.handle, comm=comm)

In [ ]:
# Load the schedule file
schedule = toast.schedule.GroundSchedule()
schedule.read(args.schedule, comm=comm)

In [ ]:
# Create a telescope for the simulation.  Again, for a specific experiment we
# would use custom classes for the site.
site = toast.instrument.GroundSite(
    schedule.site_name,
    schedule.site_lat,
    schedule.site_lon,
    schedule.site_alt,
)
telescope = toast.instrument.Telescope(
    schedule.telescope_name, focalplane=focalplane, site=site
)

In [ ]:
# Create the toast communicator.  Use the default of one group.
toast_comm = toast.Comm(world=comm)

In [ ]:
# Create the (initially empty) data
data = toast.Data(comm=toast_comm)

# Simulate data
    # ---------------------------------------------------------------

In [ ]:
# Try using site name for weather if not user-provided
weather = None
if args.weather is None:
    weather = schedule.site_name.lower()
else:
    weather = args.weather

In [ ]:
# Simulate the telescope pointing
sim_ground = toast.ops.SimGround(
    telescope=telescope,
    schedule=schedule,
    detset_key="pixel",
    weather=weather,
)
sim_ground.apply(data)

In [ ]:
# Construct a "perfect" noise model just from the focalplane parameters
default_model = toast.ops.DefaultNoiseModel()
default_model.apply(data)

In [ ]:
# Set up detector pointing.  This just uses the focalplane offsets.
det_pointing_azel = toast.ops.PointingDetectorSimple(
    boresight=sim_ground.boresight_azel, quats="quats_azel"
)
det_pointing_radec = toast.ops.PointingDetectorSimple(
    boresight=sim_ground.boresight_radec, quats="quats_radec"
)